## Youtube 2 Mp3
The aim of this notebook is to test out some functions regarding automatic youtube video downloading into mp3 files, as well as moving those files into external drives

In [1]:
import os
import subprocess
import time
import flask
import requests
import pytube

### Download the video

In [10]:
import sys

def progress_function(chunk, file_handle, bytes_remaining):
    global filesize
    current = ((filesize - bytes_remaining)/filesize)
    percent = ('{0:.1f}').format(current*100)
    progress = int(50*current)
    status = '█' * progress + '-' * (50 - progress)
    sys.stdout.write(' ↳ |{bar}| {percent}%\r'.format(bar=status, percent=percent))
    sys.stdout.flush()

def download_youtube_video_as_mp3(url):
  # create a directory called 'media' if it doesn't exist
  if not os.path.exists('media'):
    os.makedirs('media')
  
  # download the youtube video
  youtube = pytube.YouTube(url)
  video = youtube.streams.filter(only_audio=True).first()
  video_size = video.filesize
  video_path = os.path.join('media', video.default_filename)
  video.on_success = progress_function
  print('Downloading: {}'.format(video.default_filename))
  video.download('media/', filename=video.default_filename)
  
  # convert the video to mp3
  mp3_path = os.path.join('media', video.default_filename.replace('.mp4', '.mp3'))
  subprocess.run(['ffmpeg', '-i', video_path, mp3_path])
  
  # delete the original video file
  os.remove(video_path)

  # print the downloaded confirmatio
  print('Downloaded: {}'.format(video.default_filename))

  # return the file name of the mp3
  return os.path.basename(mp3_path)


In [3]:
# download_youtube_video_as_mp3('https://www.youtube.com/watch?v=NmVrnZAAklU&t=1714s')
# this works great!

### Move it to the external drive

In [4]:
import shutil

def move_file_to_external_drive(file_name):
    # get the full path of the file
    file_path = os.path.join('media', file_name)
    
    # cut the file name from the path
    file_name = os.path.basename(file_path)
    
    # define the destination path in the external drive
    external_drive_path = 'D:'
    if os.path.exists(external_drive_path):
        destination_path = os.path.join(external_drive_path, file_name)
        # move the file to the external drive
        shutil.move(file_path, destination_path)
        print(f"File '{file_name}' moved to external drive.")
    else:
        print(f"External drive not found at '{external_drive_path}'. File '{file_name}' not moved.")


In [5]:
# move_file_to_external_drive('Never Gonna Give You Up.mp3')
# it works!

# Download and move to external drive

In [12]:
def download_and_store_mp3(url=None):
    try:
        if url is None:
            url = input("Enter the YouTube video URL: ")
        mp3_file_name = download_youtube_video_as_mp3(url)
        move_file_to_external_drive(mp3_file_name)
        return mp3_file_name
    except pytube.exceptions.RegexMatchError:
        print("Invalid YouTube video URL. Please enter a valid URL.")
    except pytube.exceptions.VideoUnavailable:
        print("The requested video is unavailable. Please try again later.")
    except pytube.exceptions.ExtractError:
        print("An error occurred while extracting the video. Please try again later.")
    except pytube.exceptions.VideoPrivate:
        print("The requested video is private. Please provide a video that is not private.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [13]:
download_and_store_mp3('https://www.youtube.com/watch?v=dQw4w9WgXcQ')
#  okay its working

Downloading: Never Gonna Give You Up.mp4
Downloaded: Never Gonna Give You Up.mp4
File 'Never Gonna Give You Up.mp3' moved to external drive.


'Never Gonna Give You Up.mp3'

In [9]:
# download_and_store_mp3()

## Batch inputs
Takes a list of youtube links and downloads them into mp3 files, then moves them to the external drive

In [16]:
def download_and_store_mp3_batch(urls):
    total_files = len(urls)
    downloaded_files = 0
    for url in urls:
        try:
            mp3_file_name = download_and_store_mp3(url)
            downloaded_files += 1
            print(f"({downloaded_files}/{total_files} files downloaded)")
        except pytube.exceptions.RegexMatchError:
            print(f"Invalid YouTube video URL '{url}'. Skipping...")
        except pytube.exceptions.VideoUnavailable:
            print(f"The requested video '{url}' is unavailable. Skipping...")
        except pytube.exceptions.ExtractError:
            print(f"An error occurred while extracting the video '{url}'. Skipping...")
        except pytube.exceptions.VideoPrivate:
            print(f"The requested video '{url}' is private. Skipping...")
        except Exception as e:
            print(f"An error occurred while processing the video '{url}': {e}. Skipping...")


In [17]:
# urls = ['https://www.youtube.com/watch?v=dQw4w9WgXcQ', 'https://www.youtube.com/watch?v=ZHwVBirqD2s']
# download_and_store_mp3_batch(urls)
# it works!

Downloading: Never Gonna Give You Up.mp4
Downloaded: Never Gonna Give You Up.mp4
File 'Never Gonna Give You Up.mp3' moved to external drive.
(1/2 files downloaded)
Downloading: Im Still Standing.mp4
Downloaded: Im Still Standing.mp4
File 'Im Still Standing.mp3' moved to external drive.
(2/2 files downloaded)


In [25]:
import ipywidgets as widgets
from IPython.display import display

def download_and_store_mp3_ui():
    # create a text area for the user to input the URLs
    urls_textarea = widgets.Textarea(
        placeholder='Enter YouTube video URLs, one per line',
        description='YouTube URLs:',
        layout=widgets.Layout(width='100%', height='200px')
    )
    
    # create a button to start the download process
    download_button = widgets.Button(description='Download and store mp3 files')
    
    # create an output area to display the progress and results
    output_area = widgets.Output()
    
    # define the function to be called when the download button is clicked
    def on_download_button_clicked(b):
        # get the URLs from the text area
        urls = urls_textarea.value.split('\n')
        urls = [url for url in urls if url.strip()]  # skip empty lines
        
        # download and store the mp3 files
        download_and_store_mp3_batch(urls)
    
    # attach the on_download_button_clicked function to the download button
    download_button.on_click(on_download_button_clicked)
    
    # display the UI
    display(widgets.VBox([urls_textarea, download_button, output_area]))


In [26]:
download_and_store_mp3_ui()

Downloading: Rocket Man (Official Music Video).mp4
Downloaded: Rocket Man (Official Music Video).mp4
File 'Rocket Man (Official Music Video).mp3' moved to external drive.
(1/2 files downloaded)
Downloading: Im Still Standing.mp4
Downloaded: Im Still Standing.mp4
File 'Im Still Standing.mp3' moved to external drive.
(2/2 files downloaded)
Downloading: Rocket Man (Official Music Video).mp4
Downloaded: Rocket Man (Official Music Video).mp4
File 'Rocket Man (Official Music Video).mp3' moved to external drive.
(1/2 files downloaded)
Downloading: Im Still Standing.mp4
Downloaded: Im Still Standing.mp4
File 'Im Still Standing.mp3' moved to external drive.
(2/2 files downloaded)


In [42]:
import os
import ipywidgets as widgets
from IPython.display import display

def delete_mp3_files_ui():

    mp3_dir = 'D:'

    # get the list of mp3 files in the media directory
    mp3_files = [f for f in os.listdir('D:') if f.endswith('.mp3')]
    
    # create a checkbox for each mp3 file
    checkboxes = [widgets.Checkbox(description=f) for f in mp3_files]
    
    # create a button to delete the selected files
    delete_button = widgets.Button(description='Delete selected files')
    
    # create an output area to display the progress and results
    output_area = widgets.Output()
    
    # define the function to be called when the delete button is clicked
    def on_delete_button_clicked(b):
        # get the list of selected files
        selected_files = [checkbox.description for checkbox in checkboxes if checkbox.value]
        
        # delete the selected files
        for file_name in selected_files:
            file_path = os.path.join(mp3_dir, file_name)
            os.remove(file_path)
            print(f"File '{file_name}' deleted.")

    
    # attach the on_delete_button_clicked function to the delete button
    delete_button.on_click(on_delete_button_clicked)
    
    # display the UI
    display(widgets.VBox([widgets.Label('Select the files to delete:')] + checkboxes + [delete_button, output_area]))


In [43]:
delete_mp3_files_ui()

In [44]:
import os
import subprocess
import shutil
import pytube
import flask
import ipywidgets as widgets
from IPython.display import display

app = flask.Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if flask.request.method == 'POST':
        # handle form submission
        if 'delete_files' in flask.request.form:
            # delete selected files
            selected_files = flask.request.form.getlist('file')
            for file_name in selected_files:
                file_path = os.path.join('mp3_directory', file_name)
                os.remove(file_path)
            flask.flash(f"{len(selected_files)} files deleted.")
        elif 'search_youtube' in flask.request.form:
            # search YouTube for videos
            query = flask.request.form['query']
            videos = search_youtube(query)
            return flask.render_template('index.html', videos=videos)
        elif 'download_video' in flask.request.form:
            # download selected video
            video_url = flask.request.form['video_url']
            download_and_store_mp3(video_url)
            flask.flash("Video downloaded and stored as mp3.")
    # display the index page
    files = os.listdir('mp3_directory')
    return flask.render_template('index.html', files=files)

def search_youtube(query):
    # search YouTube for videos
    videos = []
    search_results = pytube.Search(query).results
    for result in search_results:
        if result.type == 'video':
            videos.append({'title': result.title, 'url': result.watch_url})
    return videos

def download_and_store_mp3(url):
    # create a directory called 'mp3_directory' if it doesn't exist
    if not os.path.exists('mp3_directory'):
        os.makedirs('mp3_directory')
  
    # download the youtube video
    youtube = pytube.YouTube(url)
    video = youtube.streams.filter(only_audio=True).first()
    video_size = video.filesize
    video_path = os.path.join('mp3_directory', video.default_filename)
    video.on_progress = progress_function
    print('Downloading: {}'.format(video.default_filename))
    video.download('mp3_directory/', filename=video.default_filename)
  
    # convert the video to mp3
    mp3_path = os.path.join('mp3_directory', video.default_filename.replace('.mp4', '.mp3'))
    subprocess.run(['ffmpeg', '-i', video_path, mp3_path])
  
    # delete the original video file
    os.remove(video_path)

    # print the downloaded confirmation
    print('Downloaded: {}'.format(video.default_filename))

def progress_function(stream, chunk, bytes_remaining):
    global filesize
    current = ((filesize - bytes_remaining)/filesize)
    percent = ('{0:.1f}').format(current*100)
    progress = int(50*current)
    status = '█' * progress + '-' * (50 - progress)
    sys.stdout.write(' ↳ |{bar}| {percent}%\r'.format(bar=status, percent=percent))
    sys.stdout.flush()

if __name__ == '__main__':
    app.secret_key = 'super_secret_key'
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
